## Doing spatial requests

In order to do regular spatial requests, we need the following data
 - The users location (specified in (lat,lon) format, currently set in Barcelona area)
 - The greater circle in kilometers for which we would like to limit the spatial search (two kilometers)
 - A regular user query (*:*, for example)
 - The spatial field to filter and order by (location)
 
As extra parameters, we need to tell that the Filter Query would use the **geofilt!** functionality where the greater circle is the one specified above.

Below we perform only a star-doublecolon-star query, but getting the results in 2kms distance

In [1]:
import requests

_url = "http://solr:8983/solr/retrotech/select"
_params = {
    'defType': "edismax",
    'fq': "{!geofilt}",          # filter query that applies the geofilt function given a distance 'd' 
    'q.op':"OR",
    'd':"2",                     # greater circle distance in kilometers
    'pt':"41.622393,-0.9108323", # latitude and longitude of point (Barcelona city)
    'sfield': "location",         # spatial field for filtering and sorting
    'rows': "900",
    'q':"*:*",
    'qf':"name",
    'fl':"name,location,geohash",
    'debugQuery':"true"

}

url = "http://0.0.0.0:8983/solr/retrotech/select"# ?debugQuery=true&defType="+_deftype+"&fl"+_fl+"&fq="+_fq+"&indent=true&q.op=OR&q=*:*&qf=name&rows=900&d=2&pt="+_pt+"&sfield="+_sfield+"&spatial=true"

data = requests.get(url = _url, params = _params).json()


Below list should give you list of names of the product and its latitude and longitude. Please note how we got 17 results instead of 900. This is because we are only getting results in 2 kms distance.

In [2]:
print(*((x['name'],x['location']) for x in data['response']['docs']), sep='\n')

### Suggestions time?

This same logic is the one used with the `suggest` component, that was extended with geospatial features, where the lookup implementation would factor in the geospatial score. This combination was achieved like that and with some custom joint-queries and subqueries.

I could not bring that customisation on this hands-on , sorry D: 


But for the sake of getting results over suggestions without the suggester component, we are doing a regular query to have distance-based results, with some tweaks as below 👇🏻 
 - Enlarging our greater circle radius **(to 30 kms)**
 - Doing a very recall-oriented autocomplete request, with the term **"DVD"**
 - With doing a "like" query using the Extended Dismax parser


We could boost the item titles by giving more weight to the prefixed ones, but for this example we won't do it.

In [9]:
_q="America"
# zaragoza #[41.622393,-0.9108323]
_point="41.622393,-0.9108323" # 
_url = "http://solr:8983/solr/retrotech/select"
_radius=120
_params = {
    'defType': "edismax",
    'fq': "{!geofilt}",          # filter query that applies the geofilt function given a distance 'd' 
    'q.op':"OR",
    'd':_radius,                     # greater circle distance in kilometers
    'pt':"41.622393,-0.9108323", # latitude and longitude of point
    'sfield': "location",         # spatial field for filtering and sorting
    'rows': "900",
    'q':"name:*"+_q+"*",          #ugly but working nonetheless
    'qf':"name",
    'fl':"name,location,geohash",
    'debugQuery':"true",
    #'facet': 'true',
    #'facet.field': 'name',
    #'f.names.facet.prefix':"$"+_p+"$"
}

url = "http://0.0.0.0:8983/solr/retrotech/select"# ?debugQuery=true&defType="+_deftype+"&fl"+_fl+"&fq="+_fq+"&indent=true&q.op=OR&q=*:*&qf=name&rows=900&d=2&pt="+_pt+"&sfield="+_sfield+"&spatial=true"

data = requests.get(url = _url, params = _params)
print(data.text)


{
  "responseHeader":{
    "status":0,
    "QTime":178,
    "params":{
      "q":"name:*America*",
      "defType":"edismax",
      "d":"120",
      "pt":"41.622393,-0.9108323",
      "qf":"name",
      "fl":"name,location,geohash",
      "q.op":"OR",
      "fq":"{!geofilt}",
      "sfield":"location",
      "rows":"900",
      "debugQuery":"true"}},
  "response":{"numFound":33,"start":0,"numFoundExact":true,"docs":[
      {
        "name":"American Tragedy (Edited) - CD",
        "location":"41.6521342,-0.8809428",
        "geohash":"ezrkg"},
      {
        "name":"American Graffiti - Widescreen Dubbed Subtitle Dolby - Blu-ray Disc",
        "location":"41.6521342,-0.8809428",
        "geohash":"ezrkg"},
      {
        "name":"NFL: America's Game - 1976 Oakland Raiders - Super Bowl XI - DVD",
        "location":"41.6521342,-0.8809428",
        "geohash":"ezrkg"},
      {
        "name":"American Serenade - CD",
        "location":"41.6521342,-0.8809428",
        "geohash":"ezrkg"},


Let's plot them on a map now so we can see it :D 

In [11]:
from ipyleaflet import Map, basemaps, Circle
_json = data.json()

# zaragoza #[41.622393,-0.9108323]
center = [41.622393,-0.9108323]
zoom = 8

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

for r in _json['response']['docs']:
    #(tuple(r['location'].split(',')[:-1]))
    p = Circle()
    p.radius=2
    p.weight=2
    p.color="blue"
    p.fill_color="blue"
    p.location = tuple(r['location'].split(','))
    m.add(p)
m


r = Circle()
r.radius = _radius*1000
r.weight=2
r.color = "orange"
r.fill_color = "orange"
_bcn_latlon = (41.622393,-0.9108323)
r.location = _bcn_latlon
m.add(r)
m

Map(center=[41.622393, -0.9108323], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…